# Prepare Panel

In [1]:
import os
import re
import glob
import numpy as np
import pandas as pd

In [2]:
DATA_DIR = "../../data/cleaned-data/"

In [3]:
income = pd.read_csv(DATA_DIR+"countyincome_merged.csv")
income.head()

,GEOFIPS,COUNTY,NUM_RETURN,NUM_EXEMPT,AGI,WAGES_SALARIES,DIVIDENDS,INTEREST,YEAR
0,1000,ALABAMA,1595185.0,3557069.0,39454560.0,31125750.0,638076.0,2392059.0,1989
1,1001,Autauga,13674.0,31869.0,333060.0,279230.0,3115.0,13269.0,1989
2,1003,Baldwin,39202.0,86819.0,957370.0,673040.0,24108.0,82778.0,1989
3,1005,Barbour,9367.0,21385.0,193510.0,153190.0,2729.0,12946.0,1989
4,1007,Bibb,6347.0,15087.0,129680.0,107400.0,924.0,5794.0,1989


In [4]:
gini = pd.read_csv(DATA_DIR+"census_gini_all_const.csv")
gini.head()

,STATE,COUNTY,GEOFIPS,GINI,YEAR
0,Alabama,AUTAUGA,1001,0.400131,1989.0
1,Alabama,BALDWIN,1003,0.425340,1989.0
2,Alabama,BARBOUR,1005,0.480382,1989.0
3,Alabama,BIBB,1007,0.452399,1989.0
4,Alabama,BLOUNT,1009,0.419520,1989.0


In [5]:
merged = income.merge(gini, how="inner", left_on=["YEAR", "GEOFIPS"], right_on=["YEAR", "GEOFIPS"])
merged

,GEOFIPS,COUNTY_x,NUM_RETURN,NUM_EXEMPT,AGI,WAGES_SALARIES,DIVIDENDS,INTEREST,YEAR,STATE,COUNTY_y,GINI
0,1001,Autauga,13674.0,31869.0,333060.0,279230.0,3115.0,13269.0,1989,Alabama,AUTAUGA,0.400131
1,1003,Baldwin,39202.0,86819.0,957370.0,673040.0,24108.0,82778.0,1989,Alabama,BALDWIN,0.425340
2,1005,Barbour,9367.0,21385.0,193510.0,153190.0,2729.0,12946.0,1989,Alabama,BARBOUR,0.480382
3,1007,Bibb,6347.0,15087.0,129680.0,107400.0,924.0,5794.0,1989,Alabama,BIBB,0.452399
4,1009,Blount,13270.0,31010.0,294430.0,241690.0,1984.0,14595.0,1989,Alabama,BLOUNT,0.419520
...,...,...,...,...,...,...,...,...,...,...,...,...
97267,56037,Sweetwater County,20050.0,39870.0,1461220.0,1104920.0,17977.0,8761.0,2019,Wyoming,Sweetwater County,0.413400
97268,56039,Teton County,14870.0,24920.0,4646008.0,1148792.0,862698.0,410116.0,2019,Wyoming,Teton County,0.508700
97269,56041,Uinta County,9370.0,19790.0,588952.0,452641.0,7644.0,3705.0,2019,Wyoming,Uinta County,0.367900
97270,56043,Washakie County,3740.0,7250.0,239110.0,150977.0,6117.0,3000.0,2019,Wyoming,Washakie County,0.389200


In [6]:
merged["YEAR"].unique()

array([1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
       2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], dtype=int64)

In [13]:
year_count = merged.groupby("GEOFIPS")["YEAR"].count()
valid_counties = np.array(year_count[year_count == 31].index)

In [15]:
merged_complete = merged[merged["GEOFIPS"].isin(valid_counties)]
merged_complete = merged_complete[["YEAR", "GEOFIPS", "GINI", "NUM_RETURN", "NUM_EXEMPT", "AGI", "WAGES_SALARIES", "DIVIDENDS", "INTEREST"]]
merged_complete

,YEAR,GEOFIPS,GINI,NUM_RETURN,NUM_EXEMPT,AGI,WAGES_SALARIES,DIVIDENDS,INTEREST
0,1989,1001,0.400131,13674.0,31869.0,333060.0,279230.0,3115.0,13269.0
1,1989,1003,0.425340,39202.0,86819.0,957370.0,673040.0,24108.0,82778.0
2,1989,1005,0.480382,9367.0,21385.0,193510.0,153190.0,2729.0,12946.0
3,1989,1007,0.452399,6347.0,15087.0,129680.0,107400.0,924.0,5794.0
4,1989,1009,0.419520,13270.0,31010.0,294430.0,241690.0,1984.0,14595.0
...,...,...,...,...,...,...,...,...,...
97267,2019,56037,0.413400,20050.0,39870.0,1461220.0,1104920.0,17977.0,8761.0
97268,2019,56039,0.508700,14870.0,24920.0,4646008.0,1148792.0,862698.0,410116.0
97269,2019,56041,0.367900,9370.0,19790.0,588952.0,452641.0,7644.0,3705.0
97270,2019,56043,0.389200,3740.0,7250.0,239110.0,150977.0,6117.0,3000.0
